<a href="https://colab.research.google.com/github/Meshva-p/Outlier-detection---Clinical-trails/blob/main/WeakSupervision_Snorkel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------


In [ ]:
!pip install snorkel

     |████████████████████████████████| 153kB 7.6MB/s 
     |████████████████████████████████| 1.8MB 12.4MB/s 
     |████████████████████████████████| 3.8MB 36.2MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556408 sha256=19deba09668511513564b8126f1946d0eb933e993ffdca391e7d9ee6a3a778f4
  Stored in directory: /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af
Successfully built networkx
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: networkx 2.5.1
    Uninstalling networkx-2.5.1:
      Successfully uninstalled networkx-2.5.1
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1


In [ ]:
import os
import pandas as pd
import numpy as np
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling import LFAnalysis

base_path = '/content/drive/MyDrive/flaskdata/data/NCT00703326'
model_path = '/content/drive/MyDrive/flaskdata/models/'

In [ ]:

df_adae = pd.read_csv(os.path.join(base_path, 'adae.csv'))
df_adae.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7355 entries, 0 to 7354
Columns: 115 entries, STUDYID to AEENDTC
dtypes: float64(17), object(98)
memory usage: 6.5+ MB


In [ ]:
df_null_percent = df_adae.isnull().sum()/len(df_adae)
df_null_percent.sort_values(ascending=False)[:20], df_null_percent.sort_values(ascending=False)[:20].index

(AETERM      1.000000
 AESDTH      0.998776
 AESTTPT     0.994018
 AESTRTPT    0.994018
 AESMIE      0.990891
 AESLIFE     0.987627
 AESHOSP     0.980286
 AEEDTF      0.954861
 AEOUTL      0.944799
 AESDTF      0.907274
 AEENTPT     0.787220
 AEENRTPT    0.787220
 AEACNO      0.661863
 AEACNON     0.661863
 AECONTRT    0.661863
 DEATHDT     0.346975
 ADURN       0.214140
 ADURU       0.214140
 AEEDT       0.212780
 AEEDY       0.212780
 dtype: float64,
 Index(['AETERM', 'AESDTH', 'AESTTPT', 'AESTRTPT', 'AESMIE', 'AESLIFE',
        'AESHOSP', 'AEEDTF', 'AEOUTL', 'AESDTF', 'AEENTPT', 'AEENRTPT',
        'AEACNO', 'AEACNON', 'AECONTRT', 'DEATHDT', 'ADURN', 'ADURU', 'AEEDT',
        'AEEDY'],
       dtype='object'))

##### **Dropping 90% and above missing data(null) columns**


In [ ]:
df_adae.drop(['AETERM', 'AESDTH', 'AESTTPT', 'AESTRTPT', 'AESMIE', 'AESLIFE','AESHOSP', 'AEEDTF'], axis=1, inplace=True)

##### **Dropping na in column ADURN**

In [ ]:
df_adae.dropna(subset=['ADURN'], inplace=True)

##### **Rename column ANL20FL.1 to ANL22FL per data dictionary for adae data set**

In [ ]:
df_adae.rename(columns={'ANL20FL.1': 'ANL22FL'})

,STUDYID,USUBJID,SUBJID,INVID,TRTP,TRTPN,TRTA,TRTAN,SAFFL,ICDT,TRTSDT,TRTEDT,DSTRDT,DEATHDT,COUNTRY,RACEGR1,RACEGR1N,AGEGR1,AGEGR1N,AENUM,AESEQ,AELLT,AEDECOD,AEHLT,AEHLGT,AEBODSYS,ATOXGR,ATOXGRN,AESER,AESERL,AEOUT,AEOUTL,AECONTRT,AESDT,AESDTF,AESDY,AEEDT,AEEDY,AEENRTPT,AEENTPT,...,ANL06FL,ANL07FL,ANL08FL,ANL09FL,ANL10FL,ANL11FL,ANL12FL,ANL13FL,ANL14FL,ANL15FL,ANL16FL,ANL17FL,ANL18FL,ANL19FL,ANL20FL,ANL21FL,ANL22FL,ANL23FL,ANL24FL,ANL25FL,ANL26FL,ANL27FL,ANL28FL,ANL29FL,ANL30FL,ANL31FL,ANL32FL,ANL33FL,ANL34FL,ANL35FL,ANL36FL,ANL37FL,ANL38FL,DRERRFL,LILFCFL,LILFNCFL,LBTERMFL,LIINFFL,AESTDTC,AEENDTC
0,CP12-0606,CP12-0606/1192-1207,1192-1207,INQ6663,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-19,2012-09-12,2012-12-05,2012-12-15,2013-12-14,IRL,WHITE,0.0,< 65,0.0,1.0,4.0,SINUS BRADYCARDIA,SINUS BRADYCARDIA,SUPRAVENTRICULAR ARRHYTHMIAS,CARDIAC ARRHYTHMIAS,CARDIAC DISORDERS,1.0,1.0,N,NO,UNKNOWN,NaN,NaN,2012-08-19,NaN,-24.0,2012-08-19,-24.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-08-19,2012-08-19
1,CP12-0606,CP12-0606/1192-1207,1192-1207,INQ6663,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-19,2012-09-12,2012-12-05,2012-12-15,2013-12-14,IRL,WHITE,0.0,< 65,0.0,2.0,30.0,CONSTIPATION,CONSTIPATION,GASTROINTESTINAL ATONIC AND HYPOMOTILITY DISOR...,GASTROINTESTINAL MOTILITY AND DEFAECATION COND...,GASTROINTESTINAL DISORDERS,1.0,1.0,N,NO,UNKNOWN,NaN,NaN,2012-12-18,NaN,98.0,2013-01-11,120.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-12-18,2013-01-11
2,CP12-0606,CP12-0606/1192-1207,1192-1207,INQ6663,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-19,2012-09-12,2012-12-05,2012-12-15,2013-12-14,IRL,WHITE,0.0,< 65,0.0,3.0,2.0,DIARRHEA,DIARRHOEA,DIARRHOEA (EXCL INFECTIVE),GASTROINTESTINAL MOTILITY AND DEFAECATION COND...,GASTROINTESTINAL DISORDERS,1.0,1.0,N,NO,UNKNOWN,NaN,NaN,2012-09-06,NaN,-6.0,2012-09-12,1.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-09-06,2012-09-12
3,CP12-0606,CP12-0606/1192-1207,1192-1207,INQ6663,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-19,2012-09-12,2012-12-05,2012-12-15,2013-12-14,IRL,WHITE,0.0,< 65,0.0,4.0,7.0,DIARRHEA,DIARRHOEA,DIARRHOEA (EXCL INFECTIVE),GASTROINTESTINAL MOTILITY AND DEFAECATION COND...,GASTROINTESTINAL DISORDERS,2.0,2.0,N,NO,UNKNOWN,NaN,Y,2012-09-13,NaN,2.0,2012-09-23,12.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-09-13,2012-09-23
4,CP12-0606,CP12-0606/1192-1207,1192-1207,INQ6663,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-19,2012-09-12,2012-12-05,2012-12-15,2013-12-14,IRL,WHITE,0.0,< 65,0.0,5.0,12.0,DIARRHEA,DIARRHOEA,DIARRHOEA (EXCL INFECTIVE),GASTROINTESTINAL MOTILITY AND DEFAECATION COND...,GASTROINTESTINAL DISORDERS,1.0,1.0,N,NO,UNKNOWN,NaN,NaN,2012-10-04,NaN,23.0,2012-10-05,24.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-10-04,2012-10-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7349,CP12-0606,CP12-0606/9995-5991,9995-5991,ILX6692,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2009-07-09,2009-07-19,2012-01-07,2012-02-12,2012-04-17,CAN,WHITE,0.0,< 65,0.0,39.0,146.0,SORE THROAT,OROPHARYNGEAL PAIN,UPPER RESPIRATORY TRACT SIGNS AND SYMPTOMS,RESPIRATORY DISORDERS NEC,"RESPIRATORY, THORACIC AND MEDIASTINAL DISORDERS",1.0,1.0,N,NO,UNKNOWN,NaN,NaN,2010-07-25,NaN,372.0,2010-07-28,375.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2010-07-25,2010-07-28
7351,CP12-0606,CP12-0606/9995-5991,9995-5991,ILX6692,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2009-07-09,2009

In [ ]:
df_adae['AEOUT'].isnull().sum(), df_adae['AEBODSYS'].isnull().sum(), df_adae['AGEGR1'].isnull().sum(), df_adae['ADURN'].isnull().sum()


(0, 0, 0, 0)

# Snorkel Labelling Example (on AE dataset)
- https://www.snorkel.org/use-cases/01-spam-tutorial

In [ ]:

ANOMALY = 1
NEUTRAL = 0
NORMAL = -1


@labeling_function()
def uni_var_anomaly_1(x):
    """ 
    AE Duration ADURN

    if ADURN data point is outside 3 STD
    given
        mean 49.266263
        std 127.678250
    then 
        anomaly
    """
    adurn_std = 127.678250
    adurn_mean = 49.266263
    anomaly_cut_off = adurn_std * 3.0

    lower_limit = adurn_mean - anomaly_cut_off
    upper_limit = adurn_mean + anomaly_cut_off
    
    if (x.ADURN > upper_limit) or (x.ADURN < lower_limit):
        return ANOMALY
    else:
        return NORMAL


@labeling_function()
def mul_var_anomaly_1(x):
    """ 
    refer adae_vis_2.html

    if AE outcome is RECOVERED/RESOLVED WITH SEQUELAE
        with AEBODSYS is 'GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS' and
        age is greater or equal to 65
    then 
        anomaly
    """

    if (x.AEOUT == 'RECOVERED/RESOLVED WITH SEQUELAE') and \
        (x.AEBODSYS == 'GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS') and \
        (x.AGEGR1 == '>= 65'):
        return ANOMALY
    else:
        return NORMAL


@labeling_function()
def check_neutral_1(x):
  """
  if AEOUT is UNKNOWN with AEACN1 = TREATMENT DISCONTINUED
    then NEUTRAL
  elif AEACN1 = NONE Then NORMAL
  else ANOMALY  
  """
  if x.AEOUT == 'UNKNOWN' and \
      x.AEACN1 == 'TREATMENT DISCONTINUED':
      return NEUTRAL
  elif x.AEOUT == 'UNKNOWN' and \
      x.AEACN1 == 'NONE':
      return ANOMALY
  else:
      return NORMAL


In [ ]:
from sklearn.model_selection import train_test_split

# Create a train and test split
df_train, df_test = train_test_split(
        df_adae, test_size=200, random_state=123)


In [ ]:
lfs = [uni_var_anomaly_1, mul_var_anomaly_1, check_neutral_1]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 5580/5580 [00:00<00:00, 14435.54it/s]


In [ ]:
print(np.unique(L_train))
L_train 


[-1  0  1]


array([[-1, -1, -1],
       [-1, -1,  1],
       [-1, -1,  1],
       ...,
       [-1, -1,  1],
       [-1, -1,  0],
       [-1, -1,  1]])

In [ ]:
# Find the percentage of the dataset which is label by labelling functions

coverage_uni_var_anomaly_1, coverage_mul_var_anomaly_1, coverage_check_neutral_1 = (L_train != NORMAL).mean(axis=0)

print(f"uni_var_anomaly_1 coverage: {coverage_uni_var_anomaly_1 * 100:.1f}%")
print(f"mul_var_anomaly_1 coverage: {coverage_mul_var_anomaly_1 * 100:.1f}%")
print(f"coverage_check_neutral_1 coverage: {coverage_check_neutral_1 * 100:.1f}%")


uni_var_anomaly_1 coverage: 1.5%
mul_var_anomaly_1 coverage: 0.0%
coverage_check_neutral_1 coverage: 91.7%


In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
uni_var_anomaly_1,0,[1],0.014695,0.00914,0.0
mul_var_anomaly_1,1,[],0.000000,0.00000,0.0
check_neutral_1,2,"[0, 1]",0.917204,0.00914,0.0


In [ ]:
df_train.iloc[L_train[:, 0] == ANOMALY].sample(3, random_state=2123)
# df_adae.iloc[L_train[:, 0] == ANOMALY]

,STUDYID,USUBJID,SUBJID,INVID,TRTP,TRTPN,TRTA,TRTAN,SAFFL,ICDT,TRTSDT,TRTEDT,DSTRDT,DEATHDT,COUNTRY,RACEGR1,RACEGR1N,AGEGR1,AGEGR1N,AENUM,AESEQ,AELLT,AEDECOD,AEHLT,AEHLGT,AEBODSYS,ATOXGR,ATOXGRN,AESER,AESERL,AEOUT,AEOUTL,AECONTRT,AESDT,AESDTF,AESDY,AEEDT,AEEDY,AEENRTPT,AEENTPT,...,ANL06FL,ANL07FL,ANL08FL,ANL09FL,ANL10FL,ANL11FL,ANL12FL,ANL13FL,ANL14FL,ANL15FL,ANL16FL,ANL17FL,ANL18FL,ANL19FL,ANL20FL,ANL21FL,ANL20FL.1,ANL23FL,ANL24FL,ANL25FL,ANL26FL,ANL27FL,ANL28FL,ANL29FL,ANL30FL,ANL31FL,ANL32FL,ANL33FL,ANL34FL,ANL35FL,ANL36FL,ANL37FL,ANL38FL,DRERRFL,LILFCFL,LILFNCFL,LBTERMFL,LIINFFL,AESTDTC,AEENDTC
1648,CP12-0606,CP12-0606/2989-5787,2989-5787,IFW6642,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2011-11-25,2011-12-18,2014-09-25,2014-11-05,NaN,RUS,WHITE,0.0,< 65,0.0,3.0,37.0,EDEMA OF LOWER EXTREMITIES,OEDEMA PERIPHERAL,OEDEMA NEC,GENERAL SYSTEM DISORDERS NEC,GENERAL DISORDERS AND ADMINISTRATION SITE COND...,2.0,2.0,N,NO,UNKNOWN,NaN,Y,2013-05-26,NaN,526.0,2014-10-31,1049.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2013-05-26,2014-10-31
5906,CP12-0606,CP12-0606/9415-1287,9415-1287,IWO6426,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-17,2012-08-25,2014-06-07,2014-06-28,NaN,USA,WHITE,0.0,< 65,0.0,24.0,6.0,NEUROPATHY PERIPHERAL,NEUROPATHY PERIPHERAL,PERIPHERAL NEUROPATHIES NEC,PERIPHERAL NEUROPATHIES,NERVOUS SYSTEM DISORDERS,1.0,1.0,N,NO,NOT RECOVERED/NOT RESOLVED,ONGOING,Y,2013-02-01,NaN,161.0,2014-12-31,859.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,N,2013-02-01,2014-12-31
5910,CP12-0606,CP12-0606/9415-1287,9415-1287,IWO6426,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-08-17,2012-08-25,2014-06-07,2014-06-28,NaN,USA,WHITE,0.0,< 65,0.0,28.0,9.0,VAGINAL DRYNESS,VULVOVAGINAL DRYNESS,VULVOVAGINAL SIGNS AND SYMPTOMS,VULVOVAGINAL DISORDERS (EXCL INFECTIONS AND IN...,REPRODUCTIVE SYSTEM AND BREAST DISORDERS,1.0,1.0,N,NO,NOT RECOVERED/NOT RESOLVED,ONGOING,NaN,2013-01-15,D,144.0,2014-12-31,859.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2013-01,2014-12-31


In [ ]:
# import LabelModel
from snorkel.labeling.model import LabelModel
from snorkel.labeling.model import MajorityLabelVoter

# Train the label model and compute the training labels
# cardinality is the number of the classes
label_model = LabelModel(cardinality=3, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)
label_model.save(os.path.join(model_path,'fd_snorkel_label_model_1.pt'))
# df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")


In [ ]:
L_test = applier.apply(df=df_test)
y_test_probs = label_model.predict_proba(L_test)
y_test_preds = label_model.predict(L=L_test)
# y_test_probs, y_test_preds


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 200/200 [00:00<00:00, 9836.89it/s]


In [ ]:
df_test['SNKRL_ANMLY_FL'] = y_test_preds
df_test.to_csv('/content/drive/MyDrive/flaskdata/data_output/adae_v1_snrkl.csv',index=False)
df_test.head(2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,STUDYID,USUBJID,SUBJID,INVID,TRTP,TRTPN,TRTA,TRTAN,SAFFL,ICDT,TRTSDT,TRTEDT,DSTRDT,DEATHDT,COUNTRY,RACEGR1,RACEGR1N,AGEGR1,AGEGR1N,AENUM,AESEQ,AELLT,AEDECOD,AEHLT,AEHLGT,AEBODSYS,ATOXGR,ATOXGRN,AESER,AESERL,AEOUT,AEOUTL,AECONTRT,AESDT,AESDTF,AESDY,AEEDT,AEEDY,AEENRTPT,AEENTPT,...,ANL07FL,ANL08FL,ANL09FL,ANL10FL,ANL11FL,ANL12FL,ANL13FL,ANL14FL,ANL15FL,ANL16FL,ANL17FL,ANL18FL,ANL19FL,ANL20FL,ANL21FL,ANL20FL.1,ANL23FL,ANL24FL,ANL25FL,ANL26FL,ANL27FL,ANL28FL,ANL29FL,ANL30FL,ANL31FL,ANL32FL,ANL33FL,ANL34FL,ANL35FL,ANL36FL,ANL37FL,ANL38FL,DRERRFL,LILFCFL,LILFNCFL,LBTERMFL,LIINFFL,AESTDTC,AEENDTC,SNKRL_ANMLY_FL
522,CP12-0606,CP12-0606/1836-5057,1836-5057,IIQ6692,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2011-12-30,2012-01-05,2012-05-31,2012-06-21,2013-04-18,BEL,WHITE,0.0,< 65,0.0,14.0,12.0,ANOREXIA,DECREASED APPETITE,APPETITE DISORDERS,APPETITE AND GENERAL NUTRITIONAL DISORDERS,METABOLISM AND NUTRITION DISORDERS,1.0,1.0,N,NO,UNKNOWN,NaN,NaN,2012-02-09,NaN,36.0,2012-02-16,43.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-02-09,2012-02-16,1
5788,CP12-0606,CP12-0606/9371-1854,9371-1854,IWO6436,Placebo + Docetaxel,0.0,Placebo + Docetaxel,0.0,Y,2012-07-26,2012-08-05,2012-12-30,2013-01-27,2013-07-01,USA,WHITE,0.0,< 65,0.0,3.0,18.0,WEIGHT INCREASE,WEIGHT INCREASED,PHYSICAL EXAMINATION PROCEDURES AND ORGAN SYST...,PHYSICAL EXAMINATION AND ORGAN SYSTEM STATUS T...,INVESTIGATIONS,2.0,2.0,N,NO,UNKNOWN,NaN,NaN,2012-12-09,NaN,127.0,2013-07-01,331.0,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2012-12-09,2013-07-01,1


In [ ]:

# label_dict = {1:'ANOMALY', 0:'NEUTRAL', -1:'NORMAL'}
